In [12]:
import firebase_admin
from firebase_admin import credentials, storage, db
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()


True

In [13]:
service_account_info = {
    "type": os.getenv("FIREBASE_TYPE"),
    "project_id": os.getenv("FIREBASE_PROJECT_ID"),
    "private_key_id": os.getenv("FIREBASE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("FIREBASE_PRIVATE_KEY"),
    "client_email": os.getenv("FIREBASE_CLIENT_EMAIL"),
    "client_id": os.getenv("FIREBASE_CLIENT_ID"),
    "auth_uri": os.getenv("FIREBASE_AUTH_URI"),
    "token_uri": os.getenv("FIREBASE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("FIREBASE_AUTH_PROVIDER_x509_CERT_URL"),
    "client_x509_cert_url": os.getenv("FIREBASE_CLIENT_x509_CERT_URL"),
    "universe_domain": os.getenv("FIREBASE_UNIVERSE_DOMAIN"),
}

In [ ]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred, {
    "storageBucket": "coffee-shop-d2720.firebasestorage.app",
    "databaseURL": "https://coffee-shop-d2720-default-rtdb.europe-west1.firebasedatabase.app/"
})


In [15]:
bucket = storage.bucket()

# Upload data

In [16]:
images_folder_path = "products/images"
products_collection = db.reference("products")


In [17]:
df = pd.read_json("products/products.jsonl", lines=True)
df.head()

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,Latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolat_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,Espresso_shot.webp


In [18]:
def upload_image(bucket, image_path):
    image_name = image_path.split("/")[-1]
    blob = bucket.blob(f"product_images/{image_name}")
    # upload image
    blob.upload_from_filename(image_path)
    # make access of images public + get image url
    blob.make_public()
    return blob.public_url

In [ ]:
# Remove duplicate rows based on specific columns (e.g., 'name' and 'image_path')
df = df.drop_duplicates(subset=['name', 'image_path'])

for index, row in df.iterrows():
    print(index, row["name"])
    image_path = os.path.join(images_folder_path, row["image_path"])
    image_url = upload_image(bucket, image_path)
    product_data = row.to_dict()
    product_data.pop("image_path")
    product_data["image_url"] = image_url
    # add to firebase
    products_collection.push().set(product_data)




0 Cappuccino
Added product: Cappuccino (ID: -OIfX3kIyMidaVB0tZsf)
1 Jumbo Savory Scone
Added product: Jumbo Savory Scone (ID: -OIfX4AfUW2UYs3ZmfwM)
2 Latte
Added product: Latte (ID: -OIfX4a_Te0ARX2l8lZ_)
3 Chocolate Chip Biscotti
Added product: Chocolate Chip Biscotti (ID: -OIfX4tfbziKrHBgmNbv)
4 Espresso shot
Added product: Espresso shot (ID: -OIfX5AIbtWQooLrGU2a)
5 Hazelnut Biscotti
Added product: Hazelnut Biscotti (ID: -OIfX5YLWAzZKEKEqbjX)
6 Chocolate Croissant
Added product: Chocolate Croissant (ID: -OIfX5wxoAnjbDx_ypbC)
7 Dark chocolate
Added product: Dark chocolate (ID: -OIfX6FbSIY_RXkOyHhp)
8 Cranberry Scone
Added product: Cranberry Scone (ID: -OIfX6Zxg6WaR1WNE3na)
9 Croissant
Added product: Croissant (ID: -OIfX6vMR0MY1-xLOvYG)
10 Almond Croissant
Added product: Almond Croissant (ID: -OIfX7KHRlR-3jxTT2YP)
11 Ginger Biscotti
Added product: Ginger Biscotti (ID: -OIfX7gtyFE02Ujo4CdE)
12 Oatmeal Scone
Added product: Oatmeal Scone (ID: -OIfX7yUqbArhD62axC5)
13 Ginger Scone
Added pro